In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import os
# # 设置CUDA_LAUNCH_BLOCKING环境变量为1
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# 数据预处理
transform = transforms.Compose([
    transforms.Resize((67, 67)),  # 调整图片大小
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 加载数据集
train_dataset = ImageFolder('/content/drive/MyDrive/深度学习/zibianmaqi/small_data/train', transform=transform)
test_dataset = ImageFolder('/content/drive/MyDrive/深度学习/zibianmaqi/small_data/val', transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # 输入3通道图像
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2),  # 反卷积操作
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2),  # 输出3通道图像
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# 初始化自编码器
autoencoder = Autoencoder()

criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder.to(device)

epochs = 200
all_loss = []
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, _ = data  # 不需要标签，只使用图像本身
        inputs = inputs.to(device)

        optimizer.zero_grad()

        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)  # 输入和输出之间的均方误差
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    all_loss.append(running_loss)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    running_loss = 0
plt.plot(range(len(all_loss)), all_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
all_loss.clear()


# 获取一批测试数据
dataiter = iter(test_loader)
for i in range(len(test_loader)):
    images, _ = next(dataiter)
    images = images.to(device)

    # 使用训练好的自编码器进行图像重构
    reconstructed = autoencoder(images)

    # 可视化原始图像和重构图像
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

    # 显示原始图像
    img_original = images[0].cpu().detach().permute(1, 2, 0)
    axes[0].imshow(img_original)
    axes[0].set_title('Original Image')
    axes[0].get_xaxis().set_visible(False)
    axes[0].get_yaxis().set_visible(False)

    # 显示重构图像
    img_reconstructed = reconstructed[0].cpu().detach().permute(1, 2, 0)
    axes[1].imshow(img_reconstructed)
    axes[1].set_title('Reconstructed Image')
    axes[1].get_xaxis().set_visible(False)
    axes[1].get_yaxis().set_visible(False)

    plt.show()